Packages to be used

In [15]:
import geopandas as gpd
import os
import fiona
import ipywidgets as widgets
from IPython.display import display
from rasterstats import zonal_stats
import rasterio
import rasterio.fill 
from shapely.ops import nearest_points
from earthpy import clip
import earthpy.spatial as es
import numpy as np
import tkinter as tk
from tkinter import filedialog, messagebox
import gdal
import datetime
import warnings
warnings.filterwarnings('ignore')

root = tk.Tk()
root.withdraw()
root.attributes("-topmost", True)

import json
print(datetime.datetime.now())

2020-01-22 11:20:19.586562


In [16]:
crs = 'EPSG:3395'

Define the workspace. The output layers will populate this folder. It is highly recommended to select an empty folder as your workspace
        

In [ ]:
messagebox.showinfo('OnSSET extraction', 'Output folder')
workspace = filedialog.askdirectory()

Select the clusters to be used in the analysis'

Please also idicate which column is representing the population data. 

In [17]:
messagebox.showinfo('OnSSET', 'Select the clusters')
clusters=gpd.read_file(filedialog.askopenfilename(filetypes = (("shapefile","*.shp"),("all files","*.*"))))

#popunit = widgets.Dropdown(options=clusters.head(),
#    value=None,
#    description='Population:',
#    disabled=False)

#display(popunit)

Select the administrative boundaries to be used in the analysis

In [18]:
messagebox.showinfo('OnSSET', 'Select the admin boundaries')
admin=gpd.read_file(filedialog.askopenfilename(filetypes = (("shapefile","*.shp"),("all files","*.*"))))

Select the landcover map

In [ ]:
messagebox.showinfo('OnSSET', 'Select the landcover map')
landCover=rasterio.open(filedialog.askopenfilename(filetypes = (("rasters","*.tif"),("all files","*.*"))))

clusters = zonal_stats(
    clusters,
    landCover.name,
    stats=["majority"],
    prefix="lc_", geojson_out=True, all_touched=True)
    
print(datetime.datetime.now())

Select the elevation map

In [ ]:
messagebox.showinfo('OnSSET', 'Select the elevation map')
elevation=rasterio.open(filedialog.askopenfilename(filetypes = (("rasters","*.tif"),("all files","*.*"))))

clusters = zonal_stats(
    clusters,
    elevation.name,
    stats=["mean"],
    prefix="dem_", all_touched = True, geojson_out=True)
print(datetime.datetime.now())

In [ ]:
gdal.Warp(workspace + r"\dem.tif",elevation.name,dstSRS=crs)

def calculate_slope(DEM):
    gdal.DEMProcessing(workspace + r'\slope.tif', DEM, 'slope')
    with rasterio.open(workspace + r'\slope.tif') as dataset:
        slope=dataset.read(1)
    return slope

slope=calculate_slope(workspace + r"\dem.tif")

slope = rasterio.open(workspace + r'\slope.tif')
gdal.Warp(workspace + r'\slope_4326.tif',slope.name,dstSRS='EPSG:4326')
slope_4326 = rasterio.open(workspace + r'\slope_4326.tif')

clusters = zonal_stats(
    clusters,
    slope_4326.name,
    stats=["majority"],
    prefix="sl_", all_touched = True, geojson_out=True)
print(datetime.datetime.now())

Select the ghi map

In [20]:
messagebox.showinfo('OnSSET', 'Select the ghi map')
ghi=rasterio.open(filedialog.askopenfilename(filetypes = (("rasters","*.tif"),("all files","*.*"))))

clusters = zonal_stats(
    clusters,
    ghi.name,
    stats=["mean"],
    prefix="ghi_", all_touched = True, geojson_out=True)
print(datetime.datetime.now())

2020-01-22 11:29:11.433791


Select the traveltime map

In [ ]:
messagebox.showinfo('OnSSET', 'Select the traveltime map')
travelTime=rasterio.open(filedialog.askopenfilename(filetypes = (("rasters","*.tif"),("all files","*.*"))))

clusters = zonal_stats(
    clusters,
    travelTime.name,
    stats=["mean"],
    prefix="travel_", all_touched = True, geojson_out=True)
print(datetime.datetime.now())

Select the wind velocity map

In [ ]:
messagebox.showinfo('OnSSET', 'Select the wind velocity map')
windVel=rasterio.open(filedialog.askopenfilename(filetypes = (("rasters","*.tif"),("all files","*.*"))))

clusters = zonal_stats(
    clusters,
    windVel.name,
    stats=["mean"],
    prefix="wind_", all_touched = True, geojson_out=True)
print(datetime.datetime.now())

Select the custom demand map (Optional)

In [ ]:
messagebox.showinfo('OnSSET', 'Select the custom demand map')
customDemand=rasterio.open(filedialog.askopenfilename(filetypes = (("rasters","*.tif"),("all files","*.*"))))
 
clusters = zonal_stats(
    clusters,
    customDemand.name,
    stats=["mean"],
    prefix="custom_", all_touched = True, geojson_out=True)
print(datetime.datetime.now())

Saving the clusters with extracted rasters

In [19]:
output = workspace + r'\placeholder.geojson'
with open(output, "w") as dst:
    collection = {
        "type": "FeatureCollection",
        "features": list(clusters)}
    dst.write(json.dumps(collection))
    
settlements = gpd.read_file(output)  
settlements.to_file(workspace + r"\rastersettlements.shp", driver='ESRI Shapefile')
settlements = gpd.read_file(workspace + r"\rastersettlements.shp")
proj_settlements = settlements.to_crs({ 'init': crs}) 
os.remove(output)

#def min_distance(point, lines):
#    return lines.distance(point).min()
print(datetime.datetime.now())

ValueError: Cannot write empty DataFrame to file.

Select the substations to be used in the analysis (Optional)

In [ ]:
substations=gpd.read_file(filedialog.askopenfilename(filetypes = (("shapefile","*.shp"),("all files","*.*"))))

sub_clip = clip.clip_shp(substations, admin)
sub_clip.crs = {'init' :'epsg:4326'}
sub_proj=sub_clip.to_crs({ 'init': crs})

proj_settlements['sub'] = proj_settlements.geometry.apply(lambda g: sub_proj.distance(g).min())
print(datetime.datetime.now())


Select the existing high voltage transmission lines (Optional)

In [ ]:
existing_hv=gpd.read_file(filedialog.askopenfilename(filetypes = (("shapefile","*.shp"),("all files","*.*"))))

ex_hv_clip = clip.clip_shp(existing_hv, admin)
ex_hv_clip.crs = {'init' :'epsg:4326'}
ex_hv_proj=ex_hv_clip.to_crs({ 'init': crs})

proj_settlements['ex_hv'] = proj_settlements.geometry.apply(lambda g: ex_hv_proj.distance(g).min())
print(datetime.datetime.now())

Select the planned high voltage transmission lines (Optional)


In [ ]:
planned_hv=gpd.read_file(filedialog.askopenfilename(filetypes = (("shapefile","*.shp"),("all files","*.*"))))

pl_hv_clip = clip.clip_shp(planned_hv, admin)
pl_hv_clip.crs = {'init' :'epsg:4326'}
pl_hv_proj=pl_hv_clip.to_crs({ 'init': crs})

proj_settlements['pl_hv'] = proj_settlements.geometry.apply(lambda g: pl_hv_proj.distance(g).min())
print(datetime.datetime.now())

Select the existing medium voltage distribution lines (Optional)

In [ ]:
existing_mv=gpd.read_file(filedialog.askopenfilename(filetypes = (("shapefile","*.shp"),("all files","*.*"))))

ex_mv_clip = clip.clip_shp(existing_mv, admin)
ex_mv_clip.crs = {'init' :'epsg:4326'}
ex_mv_proj=ex_mv_clip.to_crs({'init': crs})

proj_settlements['ex_mv'] = proj_settlements.geometry.apply(lambda g: ex_mv_proj.distance(g).min())
print(datetime.datetime.now())

Select the planned medium voltage distribution lines (Optional)

In [ ]:
planned_mv=gpd.read_file(filedialog.askopenfilename(filetypes = (("shapefile","*.shp"),("all files","*.*"))))

pl_mv_clip = clip.clip_shp(planned_mv, admin)
pl_mv_clip.crs = {'init' :'epsg:4326'}
pl_mv_proj=pl_mv_clip.to_crs({ 'init': crs})

proj_settlements['pl_mv'] = proj_settlements.geometry.apply(lambda g: pl_mv_proj.distance(g).min())
print(datetime.datetime.now())

Select the road network to be used (Optional)

In [ ]:
roads=gpd.read_file(filedialog.askopenfilename(filetypes = (("shapefile","*.shp"),("all files","*.*"))))

roads_clip = clip.clip_shp(roads, admin)
roads_clip.crs = {'init' :'epsg:4326'}
roads_proj=roads_clip.to_crs({ 'init': crs})

proj_settlements['roads'] = proj_settlements.geometry.apply(lambda g: roads_proj.distance(g).min())
print(datetime.datetime.now())

Select the transformers to be used (Optional)

In [ ]:
#transformers=gpd.read_file(filedialog.askopenfilename(filetypes = (("shapefile","*.shp"),("all files","*.*"))))
#trans_clip = clip.clip_shp(transformers, admin)
#trans_clip.crs = {'init' :'epsg:4326'}
#trans_proj=trans_clip.to_crs({ 'init': crs})

#proj_settlements['trans'] = proj_settlements.geometry.apply(lambda g: trans_proj.distance(g).min())
#print(datetime.datetime.now())

Select the hydropower dataset to be used (Optional)

If a hydropower dataset is selected please identify the column represeting the hydro potential as well as its unit

In [ ]:
#hydro=gpd.read_file(filedialog.askopenfilename(filetypes = (("shapefile","*.shp"),("all files","*.*"))))

hydro=gpd.read_file(r"N:\Global_Electrification_Platform\Pilot_case_Malawi\Database\Hydropower\hydro_points.shp")

#hydropower = widgets.Dropdown(options=hydro.head(),
#    value=None,
#    description='Hydropower:',
#    disabled=False)

#display(hydropower)

#hydrounit = widgets.Dropdown(options=['W', 'kW', 'MW'],
#    value='W',
#    description='Unit:',
#    disabled=False)

#display(hydrounit)
print(datetime.datetime.now())

In [ ]:
hydro_clip = clip.clip_shp(hydro, admin)
hydro_clip.crs = {'init' :'epsg:4326'}
hydro_proj=hydro_clip.to_crs({ 'init': crs})
pts3 = hydro_proj.geometry.unary_union

#x = hydropower.value
x = "PowerMW"
hydro_proj['id'] = np.arange(len(hydro_proj))

def near(point, point2=pts3):
    nearest = hydro_proj.geometry == nearest_points(point, point2)[1]
    return hydro_proj[nearest].get_values()[0]

#if hydrounit.value is 'MW':
#    proj_settlements['hydropower'] = proj_settlements.apply(lambda row: near(row.geometry)[hydro_proj.columns.get_loc(x)]*1000, axis=1)
#elif hydrounit.value is 'kW':
#    proj_settlements['hydropower'] = proj_settlements.apply(lambda row: near(row.geometry)[hydro_proj.columns.get_loc(x)], axis=1)
#else:
#    proj_settlements['hydropower'] = proj_settlements.apply(lambda row: near(row.geometry)[hydro_proj.columns.get_loc(x)]/1000, axis=1)
 
proj_settlements['hydropower'] = proj_settlements.apply(lambda row: near(row.geometry)[hydro_proj.columns.get_loc(x)]*1000, axis=1)    
proj_settlements['hydrodist'] = proj_settlements.geometry.apply(lambda g: hydro_proj.distance(g).min())
proj_settlements['hydropowerFID'] = proj_settlements.apply(lambda row: near(row.geometry)[hydro_proj.columns.get_loc('id')], axis=1)
print(datetime.datetime.now())

In [ ]:
settlements = proj_settlements.to_crs({ 'init': 'epsg:4326'}) 

settlements.to_file(workspace + r"\output.shp", driver='ESRI Shapefile')
settlements = gpd.read_file(workspace + r"\output.shp")
print(datetime.datetime.now())

In [ ]:
#settlements.rename(columns={"NightLight": "NightLights", popunit.value : "Pop", "GridCellAr":"GridCellArea"})
settlements.rename(columns={"NightLight": "NightLights", "Population" : "Pop", "Area":"GridCellArea"})

if "lc_majorit" in proj_settlements:
    settlements = settlements.rename(columns={"lc_majorit": "LandCover"})
else:
    raise Exception('The landcover column is missing, this is a mandatory column rerun cell X')
    
if "dem_mean" in proj_settlements:
    settlements = settlements.rename(columns={"dem_mean": "Elevation"})
else:
    raise Exception('The elevation column is missing, this is a mandatory column rerun cell X')    

if "sl_majorit" in proj_settlements:
    settlements = settlements.rename(columns={"sl_majorit": "Slope"})
else:
    raise Exception('The slope column is missing, this is a mandatory column rerun cell X')  

if "ghi_mean" in proj_settlements:
    settlements = settlements.rename(columns={"ghi_mean": "GHI"})
else:
    raise Exception('The solar column is missing, this is a mandatory column rerun cell X')  
    
if "travel_mea" in proj_settlements:
    settlements = settlements.rename(columns={"travel_mea": "TravelHours"})
    settlements["TravelHours"] = settlements["TravelHours"]/60
else:
    raise Exception('The traveltime column is missing, this is a mandatory column rerun cell X')
    
if "wind_mean" in proj_settlements:
    settlements = settlements.rename(columns={"wind_mean": "WindVel"})
else:
    raise Exception('The wind speed column is missing, this is a mandatory column rerun cell X')

if "custom_mea" in proj_settlements:
    settlements = settlements.rename(columns={"custom_mea": "ResidentialDemandTierCustom"})
else:
    settlements["ResidentialDemandTierCustom"] = 0

if "sub" in proj_settlements:
    settlements = settlements.rename(columns={"sub": "SubstationDist"})
    settlements["SubstationDist"] = settlements["SubstationDist"]/1000
else:
    settlements["SubstationDist"] = 99

if "ex_hv" in proj_settlements:
    settlements = settlements.rename(columns={"ex_hv": "CurrentHVLineDist"})
    settlements["CurrentHVLineDist"] = settlements["CurrentHVLineDist"]/1000  
else:
    settlements["CurrentHVLineDist"] = 0    

if "pl_hv" in proj_settlements:
    settlements = settlements.rename(columns={"pl_hv": "PlannedHVLineDist"})
    settlements["PlannedHVLineDist"] = settlements["PlannedHVLineDist"]/1000
elif 'ex_hv' not in proj_settlements:
    settlements["PlannedHVLineDist"] = 0
else:
    settlements["PlannedHVLineDist"] = settlements["CurrentHVLineDist"]  

if "ex_mv" in proj_settlements:
    settlements = settlements.rename(columns={"ex_mv": "CurrentMVLineDist"})
    settlements["CurrentMVLineDist"] = settlements["CurrentMVLineDist"]/1000
else:
    settlements["CurrentMVLineDist"] = 0

if "pl_mv" in proj_settlements:
    settlements = settlements.rename(columns={"pl_mv": "PlannedMVLineDist"})
    settlements["PlannedMVLineDist"] = settlements["PlannedMVLineDist"]/1000
elif 'ex_mv' not in proj_settlements:
    settlements["PlannedMVLineDist"] = 0
else:
    settlements["PlannedMVLineDist"] = settlements["CurrentMVLineDist"]    
    
if "roads" in proj_settlements:
    settlements = settlements.rename(columns={"roads": "RoadDist"})
    settlements["RoadDist"] = settlements["RoadDist"]/1000
else:
    settlements["RoadDist"] = 99

if "trans" in proj_settlements:
    settlements = settlements.rename(columns={"trans": "TransformerDist"})
    settlements["TransformerDist"] = settlements["TransformerDist"]/1000
else:
    settlements["TransformerDist"] = 0

if "hydropow_1" in proj_settlements:
    settlements = settlements.rename(columns={"hydropow_1": "HydropowerFID"})
else:
    settlements["HydropowerFID"] = 0

if "hydrodist" in proj_settlements:
    settlements = settlements.rename(columns={"hydrodist": "HydropowerDist"})
    settlements["HydropowerDist"] = settlements["HydropowerDist"]/1000
else:
    settlements["HydropowerDist"] = 99

if "hydropower" in proj_settlements:
    settlements = settlements.rename(columns={"hydropower": "Hydropower"})
else:
    settlements["Hydropower"] = 0

settlements["IsUrban"] = 0
settlements["PerCapitaDemand"] = 0
settlements["HealthDemand"] = 0    
settlements["EducationDemand"] = 0
settlements["AgriDemand"] = 0
settlements["CommercialDemand"] = 0
settlements["Conflict"] = 0
settlements["ElectrificationOrder"] = 0
settlements["ResidentialDemandTier1"] = 7.74
settlements["ResidentialDemandTier2"] = 43.8
settlements["ResidentialDemandTier3"] = 160.6
settlements["ResidentialDemandTier4"] = 423.4
settlements["ResidentialDemandTier5"] = 598.6
settlements["X_deg"] = settlements.geometry.centroid.x
settlements["Y_deg"] = settlements.geometry.centroid.y
print(datetime.datetime.now())

In [ ]:
settlements.to_file(workspace + r"\output.shp", driver='ESRI Shapefile')
settlements.to_file(workspace + r"\output.csv", driver='CSV')
print(datetime.datetime.now())